In [ ]:
import carla
import numpy as np
import cv2
import time
import joblib  # to load/save sklearn models
import matplotlib.pyplot as plt
import pygame
from carla import VehicleLightState

# Set up a Pygame window so we can see what’s happening
pygame.init()
width, height = 800, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("CARLA Manual Control")

# Connect to CARLA and load Town05
client = carla.Client('localhost', 2000)
client.set_timeout(25.0)
client.load_world("Town05")
world = client.get_world()

# Spawn a Tesla Model 3 somewhere it fits
blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.find('vehicle.tesla.model3')
spawn_points = world.get_map().get_spawn_points()
for spawn_point in spawn_points:
    vehicle = world.try_spawn_actor(vehicle_bp, spawn_point)
    if vehicle is not None:
        break

if vehicle is None:
    raise RuntimeError("Could not spawn vehicle — everything’s blocked.")

# Add a forward-facing RGB camera to the car
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '320')
camera_bp.set_attribute('image_size_y', '240')
camera_bp.set_attribute('fov', '110')

camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)

# This will store the latest frame from the camera
camera_image = None

# Keep track of turn signal status (-1 = left, 1 = right, 0 = none)
current_signal = 0

def process_image(image):
    """Turns CARLA’s raw image into something Pygame can display."""
    global camera_image
    # Convert to numpy array
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))
    # Drop alpha channel and switch from BGRA to RGB
    array = array[:, :, :3][:, :, ::-1]
    # Make it into a Pygame surface and scale it to fill our window
    camera_image_raw = pygame.surfarray.make_surface(array.swapaxes(0, 1))
    camera_image = pygame.transform.scale(camera_image_raw, (800, 600))
    
# A clock to control update rates if we need it
clock = pygame.time.Clock()

# Load the trained steering model
model = joblib.load("../output/baselineModel.pkl")  

def preprocess(image_array, current_signal):
    """Resize, grayscale, normalize, flatten image, and append turn signal info."""
    img = cv2.resize(image_array, (160, 120))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img / 255.0
    flat_img = img.flatten().reshape(1, -1)

    # Add signal as a separate feature
    signal_feature = np.array([[current_signal]])
    full_input = np.hstack((flat_img, signal_feature))
    return full_input

# Position the spectator camera behind and slightly above the vehicle
spectator = world.get_spectator()
transform = vehicle.get_transform()
forward_vector = transform.get_forward_vector()
cam_location = transform.location - forward_vector * 8 + carla.Location(z=3)
cam_rotation = carla.Rotation(pitch=-10, yaw=transform.rotation.yaw)
spectator.set_transform(carla.Transform(cam_location, cam_rotation))

def drive():
    """Keeps reading the camera feed, runs it through the model, and controls the car."""
    control = carla.VehicleControl()
    
    def on_image(image):
        img = np.frombuffer(image.raw_data, dtype=np.uint8)
        img = img.reshape((image.height, image.width, 4))[:, :, :3]

        # Work out which indicator is on
        current_signal = 0
        if (vehicle.get_light_state() & carla.VehicleLightState.LeftBlinker):
            current_signal = -1
        if (vehicle.get_light_state() & carla.VehicleLightState.RightBlinker):
            current_signal = 1

        X_input = preprocess(img, current_signal)
        current_signal = 0

        # Predict steering and make sure it’s within [-1, 1]
        steer = model.predict(X_input)[0]
        steer = float(np.clip(steer, -1.0, 1.0))

        # Apply control commands
        control.steer = steer
        control.throttle = 0.4
        control.brake = 0.0
        vehicle.apply_control(control)

        # Move spectator camera so it follows behind the vehicle
        transform = vehicle.get_transform()
        forward_vector = transform.get_forward_vector()
        cam_location = transform.location - forward_vector * 8 + carla.Location(z=3)
        cam_rotation = carla.Rotation(pitch=-10, yaw=transform.rotation.yaw)
        spectator.set_transform(carla.Transform(cam_location, cam_rotation))

        # Update the displayed image
        process_image(image)
        
    camera.listen(lambda image: on_image(image))

try:
    drive()

    signal_indicator = "NONE"

    while True:
        keys = pygame.key.get_pressed()

        # Control turn signals with Q, E, and R
        left_indicator_light = keys[pygame.K_q]
        right_indicator_light = keys[pygame.K_e]
        stop_indicator_light = keys[pygame.K_r]

        if left_indicator_light:
            vehicle.set_light_state(VehicleLightState.LeftBlinker)
            signal_indicator = "LEFT"
            left_indicator_light = False

        if right_indicator_light:
            vehicle.set_light_state(VehicleLightState.RightBlinker)
            signal_indicator = "RIGHT"
            right_indicator_light = False
        
        if stop_indicator_light:
            vehicle.set_light_state(VehicleLightState.NONE)
            signal_indicator = "NONE"
            stop_indicator_light = False

        # Close the window with ESC or the quit button
        for event in pygame.event.get():
            if event.type == pygame.QUIT or keys[pygame.K_ESCAPE]:
                raise KeyboardInterrupt
        
        # Draw the current turn signal status on top of the camera feed
        font = pygame.font.SysFont("Arial", 36)
        indication_lights_text = "Signal Indicator: " + signal_indicator
        indication_lights_render = font.render(indication_lights_text, True, (255, 255, 255))

        if camera_image:
            screen.blit(camera_image, (0, 0))
            screen.blit(indication_lights_render, (20, 20))
            pygame.display.flip()
        
finally:
    camera.stop()
    vehicle.destroy()
    camera.destroy()
    pygame.quit()


ModuleNotFoundError: No module named 'cv2'